In [ ]:
# remove folder from local files just for debugging purpose
# !rm -rf '/content/uncased_L-12_H-768_A-12'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 36.1MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 512kB 36.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e1ea7837d227ee9c6ca1c5d4657e846cae347e5ed0e6c1c424d7f4d5197ac5c5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/SuicideAndDepression_Detection.csv')
train.shape

(348124, 2)

In [ ]:
train.head()

,text,class
0,Does life actually work for most / non-depress...,depression
1,I found my friend's bodyIt was almost nine yea...,depression
2,Ex Wife Threatening SuicideRecently I left my ...,SuicideWatch
3,Am I weird I don't get affected by compliments...,teenagers
4,Finally 2020 is almost over... So I can never ...,teenagers


In [ ]:
import re
import string
from tqdm.notebook import tqdm


# clean text from noise
def clean_text(text):
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # taking sequences of characters with alphanumeric characters separated by other characters
    text = re.sub(r"[-?!&]",' ',text)
    text = re.sub(r'''["#$%()*+,./:;<=>@[\]^_`{|}~]''','',text)
    
    # remove all numeric charracters
    text = re.sub(r'[0-9]','',text)

    text = re.sub(r"\s+"," ", text)

    Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
           "'d":" would","'ve":" have","'re":" are"}
  
    #replace the contractions
    for key,value in Apos_dict.items():
        if key in text:
            text=text.replace(key,value)

    # convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

text = '''heel!! my9 99 name is daygl? 

    again keep it     rea@l doesn't non-task me th're they've '''
print(clean_text(text))

for i in tqdm(range(348124)):
    try:
        train['text'][i] = clean_text(train['text'][i])
    except:
        print(i)
        train = train.drop(i)
    i+=1

heel my name is daygl again keep it real does not non task me th are they have 


185323



In [ ]:
train.shape

(348123, 2)

In [ ]:
import nltk
from tqdm.notebook import tqdm
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')

for i in tqdm(range(348123)):
    try:
        train['text'][i] = ' '.join([word for word in train['text'][i].split() if word not in (stop)])
    except:
        pass

# uncomment this line to do the same task in one line. I just wanted to keep track of the percentage completion.    
# train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


import nltk

lemmatizer = nltk.stem.WordNetLemmatizer()

for i in tqdm(range(348123)):
    try:
        train['text'][i] = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in train['text'][i].split()])
    except:
        pass


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
train.head()

,text,class
0,life actually work non depressed people seem p...,depression
1,found friend bodyit almost nine year ago still...,depression
2,ex wife threaten suiciderecently left wife goo...,SuicideWatch
3,weird get affected compliment come someone kno...,teenagers
4,finally almost never hear bad year ever swear ...,teenagers


In [ ]:
import progressbar
import nltk
nltk.download('punkt')

i = 0
max_tokens = 0
avg_tokens = 0
data_size = len(train['text'])
bar = progressbar.ProgressBar(maxval=data_size, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

bar.start()
for text in train['text']:
    nltk_tokens = nltk.word_tokenize(text)
    if (len(nltk_tokens) > max_tokens):
        max_tokens = len(nltk_tokens)
    avg_tokens+=(len(nltk_tokens)/data_size)

    i+=1
    bar.update(i)

bar.finish()

print(max_tokens)
print(avg_tokens)

[                                                                        ] N/A%

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[========================================================================] 100%


5850
78.8291580849239


In [ ]:
train = train.rename(columns = {'class' : 'labels'})
train.head()

,text,labels
0,life actually work non depressed people seem p...,depression
1,found friend bodyit almost nine year ago still...,depression
2,ex wife threaten suiciderecently left wife goo...,SuicideWatch
3,weird get affected compliment come someone kno...,teenagers
4,finally almost never hear bad year ever swear ...,teenagers


In [ ]:
train.to_csv('/content/gdrive/MyDrive/Colab Notebooks/preProcessedSuicideAndDepression_Detection.csv', index = False)

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/preProcessedSuicideAndDepression_Detection.csv')

In [ ]:
train.dropna(inplace=True)
train.head()

,text,labels
0,life actually work non depressed people seem p...,depression
1,found friend bodyit almost nine year ago still...,depression
2,ex wife threaten suiciderecently left wife goo...,SuicideWatch
3,weird get affected compliment come someone kno...,teenagers
4,finally almost never hear bad year ever swear ...,teenagers


In [ ]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train.text, train.labels, test_size=0.25, random_state=42)

print('X_train shape:',X_train.shape)
print('X_train shape:',X_val.shape)

X_train shape: (261018,)
X_train shape: (87006,)


In [ ]:
X_train

183834    wont get betteri wont get well stop stay im po...
316057             watch futurama thought id date amy leela
28161     suggestion feel worthless overwhelmed like dro...
177238    really really v frustrate time brain decides s...
331116    anyone ever get goosebump shit idk hate make b...
                                ...                        
119918    even sure sayhello everyone sure many people s...
259255        hey wanna know something regret know bout pee
131975    ghost problem againi start feel half scar half...
146914    know girlthis first time make post go nothing ...
121998    school could go train feel well myselfbut woul...
Name: text, Length: 261018, dtype: object

In [ ]:
X_val

182258    fuck chemistry chemistry honestly go fuck hole...
188097    b day dont know guy care last birth day teenag...
221083    one read thisi get bad bad try commit suicide ...
309503    found resource make call baby step right httpw...
317025    random text post accident come today day hour ...
                                ...                        
158345    someone depression describe like do want know ...
180634    feel lonely confusedalright really know start ...
90579     im fuckedim lose mind im year old go cant list...
125675    sick work every job shitshow unemployed ca bri...
53591     wise word man help need depressed always expec...
Name: text, Length: 87006, dtype: object

In [ ]:
pip install -U bert-serving-server bert-serving-client

     |████████████████████████████████| 71kB 4.1MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=0d1a6628aebd605596911d5d8199d89515d5efb5bbb6b94260e74b0f113bc916
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip && unzip uncased_L-12_H-768_A-12.zip

--2021-05-17 08:23:16--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 142.250.98.128, 142.250.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   201MB/s    in 1.9s    

2021-05-17 08:23:18 (201 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!nohup bert-serving-start -model_dir uncased_L-12_H-768_A-12/ -num_worker=2 -max_seq_len 512 > out.file 2>&1 &

In [ ]:
from bert_serving.client import BertClient

# make a connection with the BERT server using it's ip address
bc = BertClient()
# get the embedding for train and val sets
# get the embedding
embedding = bc.encode(['First do it', 'then do it right', 'then do it better'])
# check the shape of embedding, it should be 3x768
print(embedding)
print(embedding.shape)


KeyboardInterrupt: ignored

In [ ]:
from tqdm.notebook import tqdm

X_train_bert = []

for i in tqdm(range(261018)):
    temp = bc.encode(X_train[i:i+1].to_list())
    X_train_bert.append(temp[0])
X_train_bert = np.array(X_train_bert)

import pickle

with open('/content/gdrive/MyDrive/Colab Notebooks/trainSuicideAndDepression_DetectionBERTWordEmbeddings.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(X_train_bert, filehandle)


/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=512" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [ ]:
X_val_bert = []

for i in tqdm(range(87006)):
    temp = bc.encode(X_val[i:i+1].to_list())
    X_val_bert.append(temp[0])
X_val_bert = np.array(X_val_bert)

with open('/content/gdrive/MyDrive/Colab Notebooks/testSuicideAndDepression_DetectionBERTWordEmbeddings.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(X_val_bert, filehandle)

/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=512" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [ ]:
import pickle
with open('/content/gdrive/MyDrive/Colab Notebooks/trainSuicideAndDepression_DetectionBERTWordEmbeddings.data', 'rb') as filehandle:
    # read the data as binary data stream
    X_train_bert = pickle.load(filehandle)

with open('/content/gdrive/MyDrive/Colab Notebooks/testSuicideAndDepression_DetectionBERTWordEmbeddings.data', 'rb') as filehandle:
    # store the data as binary data stream
    X_val_bert = pickle.load(filehandle)

In [ ]:
print(X_train_bert)

[[-0.3179774   0.17453584  0.5170291  ... -0.7881806  -0.25709733
   0.00385739]
 [ 0.24498028 -0.06635986  0.12903148 ... -0.47168267  0.05232501
  -0.19207843]
 [-0.3478057   0.49102622  0.81172705 ... -0.8407974  -0.49174997
  -0.18910503]
 ...
 [-0.3070692   0.2546062   0.7424636  ... -0.7569532  -0.30264077
  -0.01356569]
 [-0.5746188   0.08246285  0.7418346  ... -0.5503757  -0.10338228
  -0.35683793]
 [ 0.25714302  0.6005804   0.5426199  ... -0.7847218  -0.38745683
  -0.29998747]]


In [ ]:
print(X_val_bert.shape)
print(X_val_bert)

(87006, 768)
[[ 0.585339    1.0363818   0.64045656 ... -0.6225818  -0.13340841
  -0.34627154]
 [-0.12839574  0.22486146  0.87340164 ... -1.2995867  -0.19449522
  -0.633964  ]
 [-0.46718404  0.26355556  0.6910244  ... -1.0066607  -0.41251537
  -0.34015918]
 ...
 [-0.28903404  0.63441694  0.6014907  ... -0.84442043 -0.39032087
  -0.21951003]
 [-0.1503326   0.26507553  0.6809735  ... -0.6676791  -0.24228257
  -0.3163622 ]
 [-0.09856147  0.01743699  0.9631243  ... -0.67891777 -0.41024834
   0.10065776]]


In [ ]:
#X_train_bert = bc.encode(X_train[:110].tolist())
#X_val_bert = bc.encode(X_val[:10].tolist())
#X_train_bert

In [ ]:
from sklearn import preprocessing

scaler_Xtrain_bert = preprocessing.StandardScaler().fit(X_train_bert)
scaler_Xval_bert = preprocessing.StandardScaler().fit(X_val_bert)


#scaler_Xtrain_bert.mean_
#scaler_Xval_bert.mean_

#scaler_Xtrain_bert.scale_
#scaler_Xval_bert.scale_

X_train_bert_scaled = scaler_Xtrain_bert.transform(X_train_bert)
X_val_bert_scaled = scaler_Xval_bert.transform(X_val_bert)

In [ ]:
X_train_bert_scaled

array([[-0.5781883 , -0.36921924, -0.8479416 , ...,  0.01601481,
         0.2160719 ,  1.233294  ],
       [ 1.2383155 , -1.2568494 , -2.6822171 , ...,  1.1706452 ,
         1.7553034 ,  0.3330605 ],
       [-0.67443573,  0.79695517,  0.54525566, ..., -0.17593905,
        -0.95121527,  0.34672186],
       ...,
       [-0.5429907 , -0.07418346,  0.21780968, ...,  0.12993693,
        -0.01048537,  1.1532432 ],
       [-1.4062968 , -0.70848125,  0.21483602, ...,  0.8835617 ,
         0.98073244, -0.4239324 ],
       [ 1.2775612 ,  1.2006302 , -0.72696006, ...,  0.028633  ,
        -0.43240568, -0.1627311 ]], dtype=float32)

In [ ]:
X_val_bert_scaled

array([[ 2.3577516 ,  2.8080757 , -0.26532012, ...,  0.6189567 ,
         0.83341366, -0.37093085],
       [ 0.03863893, -0.18141381,  0.83907413, ..., -1.8393143 ,
         0.52943325, -1.682963  ],
       [-1.0621737 , -0.03887198, -0.02557751, ..., -0.7756715 ,
        -0.55547976, -0.34305516],
       ...,
       [-0.48331735,  1.327312  , -0.4500576 , ..., -0.18656121,
        -0.44503537,  0.20716976],
       [-0.03263973, -0.03327267, -0.07322914, ...,  0.455204  ,
         0.29163358, -0.23452818],
       [ 0.13557835, -0.9455268 ,  1.2644497 , ...,  0.4143952 ,
        -0.5441985 ,  1.667307  ]], dtype=float32)

In [ ]:
from sklearn.linear_model import LogisticRegression
import time

# LR model
# make dual = True for solver = 'liblinear' 
model_bert = LogisticRegression(dual=False, max_iter=1000, solver='lbfgs')

# train
start = time.time()                                
model_bert = model_bert.fit(X_train_bert_scaled, y_train)
end = time.time()

# predict
pred_bert = model_bert.predict(X_val_bert_scaled)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))
print(end-start)

0.7824977587752568
397.81691122055054
